## Ввод датасета/нашей документации

In [1]:
from striprtf.striprtf import rtf_to_text

In [2]:
file=open('001.rtf', 'r')

In [3]:
rtfdoc=file.read()

In [4]:
document=rtf_to_text(rtfdoc)

In [5]:
document

'\nНаименование документа|ИНСТРУКЦИЯ "АГЕНТА КОММЕРЧЕСКОГО"|\nВид документа|ИНСТРУКЦИЯ|\nДолжностная инструкция|АГЕНТ КОММЕРЧЕСКИЙ|\nОглавление|ОБРАЗЕЦ ДОЛЖНОСТНОЙ ИНСТРУКЦИИ\n     КОММЕРЧЕСКОЕ ПОСРЕДНИЧЕСТВО\n          ТЕХНИЧЕСКИЕ ИСПОЛНИТЕЛИ|\n\n\nКомментарии\nТекст документа\n\nИНСТРУКЦИЯ\n\nАГЕНТА КОММЕРЧЕСКОГО\n\nДОЛЖНОСТНАЯ ИНСТРУКЦИЯ|УТВЕРЖДАЮ|\n||\n|(орган юридического лица (учредители);|\n00.00.00  № 00|лицо, уполномоченное утверждать|\nагента коммерческого|должностную инструкцию)|\n||||\n|(подпись)||(фамилия, инициалы)|\n|00.00.00 г.|||\n\n\nI. Общие положения\n\n1. Коммерческий агент относится к категории технических исполнителей.\n2. На должность коммерческого агента назначается лицо, имеющее среднее профессиональное образование и специальная подготовка по установленной программе без предъявления требований к стажу работы.\n3. Назначение на должность коммерческого агента и освобождение от нее производится приказом директора предприятия.\n4. Коммерческий агент должен знать:\

## Готовим инструменты

In [6]:
import re
import logging

import gensim.downloader as api
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import pymorphy2

#from nltk import pos_tag_sents
#from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk_download('averaged_perceptron_tagger_ru')
#nltk.download('universal_tagset')
#pos_tag_sents(word_tokenize("настоящей должностной инструкцией"), tagset='universal',lang='rus')

## Подключаем gensim и загружаем векторную модель слов (уже готовую, model.model)

In [13]:
import gensim

In [7]:
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [9]:
api.BASE_DIR='C:\\Users\\Home\\gensim-data\\'

In [10]:
api.BASE_DIR

'C:\\Users\\Home\\gensim-data\\'

In [11]:
#model=api.load("word2vec-ruscorpora-300")

In [15]:
md=gensim.models.KeyedVectors.load("C:\\Users\\Home\\gensim-data\\model.model") #C:\\Users\\Home\\gensim-data\\model.model

INFO - 13:58:52: loading Word2VecKeyedVectors object from C:\Users\Home\gensim-data\model.model
INFO - 13:58:52: loading vectors from C:\Users\Home\gensim-data\model.model.vectors.npy with mmap=None
INFO - 13:58:53: loading vectors_ngrams from C:\Users\Home\gensim-data\model.model.vectors_ngrams.npy with mmap=None
INFO - 13:58:59: loading vectors_vocab from C:\Users\Home\gensim-data\model.model.vectors_vocab.npy with mmap=None
INFO - 13:58:59: setting ignored attribute vectors_ngrams_norm to None
INFO - 13:58:59: setting ignored attribute vectors_norm to None
INFO - 13:58:59: setting ignored attribute buckets_word to None
INFO - 13:58:59: setting ignored attribute vectors_vocab_norm to None
INFO - 13:58:59: loaded C:\Users\Home\gensim-data\model.model


In [21]:
md.most_similar(positive=['главный', 'бухгалтер'], topn=15)

[('бухгалтерский', 0.7341619729995728),
 ('главноуправляющий', 0.697599470615387),
 ('-главный', 0.6931504011154175),
 ('.главный', 0.6719357371330261),
 ('неглавный', 0.6708090305328369),
 ('главбух', 0.6618990302085876),
 ('бухгалтерско', 0.6565037965774536),
 (',главный', 0.6553462743759155),
 ('главбухша', 0.6491634845733643),
 ('менеджерский', 0.6470004916191101),
 ('бухгалтерия', 0.6442776918411255),
 ('-администратор', 0.6385512351989746),
 ('наиглавный', 0.6352169513702393),
 ('офис-менеджер', 0.6255849599838257),
 ('глав-ный', 0.6252533197402954)]

## Получаем входной текст и предобрабатываем его: леммтизация, токенизация, выкидывание стоп-слов, частых/редких слов

In [46]:
init_text="Продажи и покупки, оформление документов, оформление договоров."

In [42]:
stop_words = stopwords.words('russian')

In [47]:
init_text=init_text.lower()

### Разбиваем текст на отдельные токены (здесь, по сути, на слова)

In [ ]:
tokenizer = RegexpTokenizer('\w+|[^\w\s]+\.') # с паттернами сложно, надо отрегулировать #'\w+|[^\w\s]+'

In [48]:
tokens=tokenizer.tokenize(init_text)

In [51]:
tokens

['продажи',
 'и',
 'покупки',
 'оформление',
 'документов',
 'оформление',
 'договоров']

### Выкидываем слова, не несущие смысловой нагрузки ("стоп-слова")

In [52]:
tokens_clear=[]
for t in tokens:
    if t in stop_words:
        continue
    tokens_clear.append(t)

In [53]:
tokens_clear

['продажи', 'покупки', 'оформление', 'документов', 'оформление', 'договоров']

### Лемматизируем (берем начальную форму)

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [54]:
tokens_lemmatized=[]
for t in tokens_clear:
    p = morph.parse(t)[0]
    p=p.normal_form
    tokens_lemmatized.append(p)

In [55]:
tokens_lemmatized

['продажа', 'покупка', 'оформление', 'документ', 'оформление', 'договор']

### Строго говоря, есть подводные камни (например, "оформление" - > "оформлять"?), но это уже тонкая настройка

### Получили обработанный вводный текст

### Оформим в виде функции

In [56]:
def prepare_text(init_text, tokenizer=tokenizer, stop_words=stop_words, morpher=morph):
    init_text=init_text.lower()
    tokens=tokenizer.tokenize(init_text)
    tokens_clear=[]
    for t in tokens:
        if t in stop_words:
            continue
        tokens_clear.append(t)
    tokens_lemmatized=[]
    for t in tokens_clear:
        p = morph.parse(t)[0]
        p=p.normal_form
        tokens_lemmatized.append(p)
    return tokens_lemmatized

## Далее следует ввод данных из датасета. Пока это символические несколько параграфов